In [17]:
import pandas as pd  # For data manipulation and reading CSV files
import numpy as np  # For numerical computations
from sklearn.linear_model import LogisticRegression  # Logistic Regression model
from sklearn.model_selection import cross_val_score  # Cross-validation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Evaluation metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [26]:

train_data = pd.read_csv("/home/tddy/ML/Final project/html-2024-fall-final-project-stage-1/train_data.csv")  
# Drop rows with missing values in the specified columns

#fill Nan
numerical_columns = train_data.select_dtypes(include=['number']).columns  # Select only numeric columns
train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].mean())

# One-hot encode 'home_team_season' and 'away_team_season'
original_columns = train_data.columns.tolist()

#one hot encode
onehot_columns = ['home_team_season', 'away_team_season','home_team_abbr','away_team_abbr','season', ]
train_data = pd.get_dummies(train_data, columns=onehot_columns, drop_first=True)

# Save the new column names after one-hot encoding
new_columns = train_data.columns.tolist()

# Identify the newly added columns
dummy_columns = [col for col in new_columns if col not in original_columns]

print(dummy_columns)



# #fill train_data
# new_columns = pd.DataFrame({
#     'team_rest_diff': train_data['home_team_rest'] - train_data['away_team_rest'],
#     'pitcher_rest_diff': train_data['home_pitcher_rest'] - train_data['away_pitcher_rest'],
#     'batting_avg_diff': train_data['home_batting_batting_avg_10RA'] - train_data['away_batting_batting_avg_10RA'],
#     'pitching_ERA_diff': train_data['home_pitching_earned_run_avg_10RA'] - train_data['away_pitching_earned_run_avg_10RA'],
#     'onbase_perc' : train_data['home_batting_onbase_plus_slugging_10RA'] / train_data['away_batting_onbase_plus_slugging_10RA']
# })

# Concatenate the new columns to the original DataFrame
# train_data = pd.concat([train_data, new_columns], axis=1)
columns_to_drop = ['away_pitcher', 'home_pitcher', 'date','id']
train_data.drop(columns=columns_to_drop, inplace=True)
train_data['is_night_game'] = train_data['is_night_game'].apply(
    lambda x: np.random.choice([True, False]) if pd.isnull(x) else x
)

# Example of splitting the dataset by winning and losing teams
# Assuming 'home_team_win' is the column that indicates the outcome (1 = win, 0 = loss)
# winning_team_data = train_data[train_data['home_team_win'] == 1]
# losing_team_data = train_data[train_data['home_team_win'] == 0]
# numeric_columns = train_data.select_dtypes(include=['number']).columns
# # Loop through all columns and perform a t-test for each column
# count = 0
# add_colums=[]
# for column in numeric_columns:
#     if column != 'home_team_win':  # Skip the outcome column itself
#         # Perform t-test (assuming data is numeric)
#         t_stat, p_value = stats.ttest_ind(winning_team_data[column], losing_team_data[column], nan_policy='omit')
        
#         # If p-value is less than 0.05, print the result
#         if p_value < 0.05:
#             # print(f"Significant variable: {column} | p-value: {p_value}")
#             add_colums.append(column)

# print(train_data)
y = train_data['home_team_win']
X = train_data.drop(columns=['home_team_win'])

trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=27)


['home_team_season_ARI_2017', 'home_team_season_ARI_2018', 'home_team_season_ARI_2019', 'home_team_season_ARI_2021', 'home_team_season_ARI_2022', 'home_team_season_ARI_2023', 'home_team_season_ATL_2016', 'home_team_season_ATL_2017', 'home_team_season_ATL_2018', 'home_team_season_ATL_2019', 'home_team_season_ATL_2021', 'home_team_season_ATL_2022', 'home_team_season_ATL_2023', 'home_team_season_BAL_2016', 'home_team_season_BAL_2017', 'home_team_season_BAL_2018', 'home_team_season_BAL_2019', 'home_team_season_BAL_2021', 'home_team_season_BAL_2022', 'home_team_season_BAL_2023', 'home_team_season_BOS_2016', 'home_team_season_BOS_2017', 'home_team_season_BOS_2018', 'home_team_season_BOS_2019', 'home_team_season_BOS_2021', 'home_team_season_BOS_2022', 'home_team_season_BOS_2023', 'home_team_season_CHC_2016', 'home_team_season_CHC_2017', 'home_team_season_CHC_2018', 'home_team_season_CHC_2019', 'home_team_season_CHC_2021', 'home_team_season_CHC_2022', 'home_team_season_CHC_2023', 'home_team_se

In [ ]:
bagging_rbf_svm = BaggingClassifier(
    estimator=SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=27),
    n_estimators=10,  # Number of base estimators
    max_samples=0.8,  # Sample size for each base estimator
    max_features=0.8,  # Features used for each base estimator
    bootstrap=True,   # Sample with replacement
    bootstrap_features=True,  # Sample features
    random_state=27
)
bagging_linear_svm = BaggingClassifier(
    estimator=LinearSVC(C=1.0,random_state=27),
    n_estimators=10,
    max_samples=0.8,
    max_features=0.8,
    bootstrap=True,
    bootstrap_features=True,
    random_state=27
)
# bagging_poly_svm = BaggingClassifier(
#     base_estimator=SVC(kernel='poly', degree=2, probability=True),
#     n_estimators=10,
#     max_samples=0.8,
#     max_features=0.8,
#     bootstrap=True,
#     bootstrap_features=True,
#     random_state=42
# )

bagging_linear_svm.fit(trainX,trainy)
bagging_rbf_svm.fit(trainX,trainy)

linear_pred=bagging_linear_svm.predict_proba(trainX)
rbf_pred=bagging_rbf_svm.predict_proba(trainX)
blended_train_pred = (rbf_pred + linear_pred) / 2
    
# Train a meta-classifier on blended predictions
meta_classifier = LogisticRegression(random_state=27)
meta_classifier.fit(blended_train_pred, trainy)

# Predict on test set
rbf_test_pred = bagging_rbf_svm.predict_proba(trainX)
linear_test_pred = bagging_linear_svm.predict_proba(trainX)
blended_test_pred = (rbf_test_pred + linear_test_pred) / 2

# Final predictions using meta-classifier
final_predictions = meta_classifier.predict(blended_test_pred)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(testy, final_predictions)
report = classification_report(testy, final_predictions)

print(
    'rbf_bagging_svm:', bagging_rbf_svm,
    'linear_bagging_svm:', bagging_linear_svm,
    'meta_classifier:', meta_classifier,
    'accuracy:', accuracy,
    'classification_report:', report
)


In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['rbf', 'linear', 'poly']
}

grid_search = GridSearchCV(
    SVC(), 
    param_grid, 
    cv=5,  # 5-fold cross-validation
    scoring='accuracy', 
    verbose=3, 
    n_jobs=-1  # Use all available cores
)
grid_search.fit(all_data_x, all_data_y)

results = grid_search.cv_results_
for i in range(len(results['params'])):
    print(f"Combination: {results['params'][i]}, Mean CV Accuracy: {results['mean_test_score'][i]:.4f}")

In [ ]:
# param_grid = {
#     'kernel': ['poly'],  # Focus on polynomial kernel
#     'degree': [2, 3, 4, 5],  # Different polynomial degrees
#     'C': [0.1, 1, 10, 100],  # Regularization parameter
#     'coef0': [0.0, 0.1, 1],  # Independent term in kernel function
#     'gamma': ['scale', 'auto', 0.1, 1]  # Kernel coefficient
# }

# grid_search = GridSearchCV(
#     SVC(), 
#     param_grid, 
#     cv=5,  # 5-fold cross-validation
#     scoring='accuracy',
#     n_jobs=-1,  # Use all available cores
#     verbose=1
# )

grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-validation Score:", grid_search.best_score_)

# Best model
best_svm = grid_search.best_estimator_

Fitting 5 folds for each of 192 candidates, totalling 960 fits


KeyboardInterrupt: 